In [9]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from datetime import datetime
import dateutil
import numpy as np


crypto = {
    'yahoo' : 'https://finance.yahoo.com/crypto/',
    'cmc' : 'https://coinmarketcap.com/'
}

def scrape_yahoo():
    data = requests.get(crypto['yahoo']).text
    soup = BeautifulSoup(data,'html.parser')

    # find tables on the wiki page
    for table in soup.find_all('table'):
        print(table.get('class'))

    tables = soup.find_all('table')
    table = soup.find('table', class_='W(100%)')

    col_to_scrape=[
        'Symbol', 
        'Name', 
        'Price (Intraday)', 
        'Change', 
        '% Change', 
        'Market Cap', 
        'Volume in Currency (Since 0:00 UTC)', 
        'Volume in Currency (24Hr)',
        'Total Volume All Currencies (24Hr))',
        'Circulating Supply',
        'created_at'
    ]

    # create empty dataframe with column names
    df = pd.DataFrame(columns=col_to_scrape)

    #scrape data from site
    for row in table.tbody.find_all('tr'):
        # Find all data for each column
        columns = row.find_all('td')
        
        if(columns != []):
            sym = columns[0].text.strip()
            name = columns[1].text.strip()
            price = columns[2].text.strip()
            chge = columns[3].text.strip()
            chge_percent = columns[4].text.strip()
            mkt_cap = columns[5].text.strip()
            vol_utc = columns[6].text.strip()
            vol_24hr = columns[7].text.strip()
            total_vol = columns[8].text.strip()
            cir_supply = columns[9].text.strip()

            scraped_values = [
                sym,
                name, 
                price, 
                chge, 
                chge_percent,
                mkt_cap,
                vol_utc,
                vol_24hr,
                total_vol,
                cir_supply,
                datetime.now()
                ]

            df = df.append({item[0]: item[1] for item in zip(col_to_scrape,scraped_values)}, ignore_index=True)

    return df.sample(20)

scrape_yahoo()

['W(100%)']


,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr)),Circulating Supply,created_at
4,USDC-USD,USD Coin USD,0.999987,-0.000123,-0.01%,42.461B,3.767B,3.767B,3.767B,42.462B,2023-03-01 10:52:04.784344
5,XRP-USD,XRP USD,0.382364,+0.001416,+0.37%,19.482B,947.933M,947.933M,947.933M,50.951B,2023-03-01 10:52:04.787759
21,LINK-USD,Chainlink USD,7.4318,+0.0753,+1.02%,3.775B,310.235M,310.235M,310.235M,508M,2023-03-01 10:52:04.833511
18,STETH-USD,Lido Staked ETH USD,"1,635.99",+18.85,+1.17%,5.706B,7.947M,7.947M,7.947M,3.488M,2023-03-01 10:52:04.825152
20,UNI7083-USD,Uniswap USD,6.7989,+0.2232,+3.39%,5.182B,97.909M,97.909M,97.909M,762.209M,2023-03-01 10:52:04.830067
15,SHIB-USD,Shiba Inu USD,0.000012,-0.000000,-0.98%,6.728B,177.597M,177.597M,177.597M,549.063T,2023-03-01 10:52:04.817036
19,DAI-USD,Dai USD,0.999847,-0.000256,-0.03%,5.209B,183.69M,183.69M,183.69M,5.21B,2023-03-01 10:52:04.827538
7,ADA-USD,Cardano USD,0.360942,-0.000890,-0.25%,12.514B,268.028M,268.028M,268.028M,34.671B,2023-03-01 10:52:04.793100
22,ATOM-USD,Cosmos USD,12.73,+0.22,+1.75%,3.647B,126.217M,126.217M,126.217M,286.37M,2023-03-01 10:52:04.838438
1,ETH-USD,Ethereum USD,"1,655.43",+13.87,+0.84%,202.582B,8.018B,8.018B,8.018B,122.374M,2023-03-01 10:52:04.774658


In [14]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

# Set up the model and prompt
model_engine = "text-davinci-003"
prompt = "Hello, how are you today?"

# Generate a response
completion = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=0.5,
)

response = completion.choices[0].text
print(response)



I'm doing well, thank you. How about you?
